In [1]:
import pandas as pd
import numpy as np

In [2]:
lawyers_cases = pd.DataFrame.from_csv("../data/advocate_data/lawyers_cases_1946-2014.csv")

In [3]:
lawyers_cases['side'] = "NaN"
lawyers_cases.loc[(lawyers_cases["petitioner_x"] == 1) & (lawyers_cases["petitioner_y"] == 1), "side"] = 'p'
lawyers_cases.loc[(lawyers_cases["petitioner_x"] == 1) & (lawyers_cases["petitioner_y"].isnull()), "side"] = 'p'
lawyers_cases.loc[(lawyers_cases["petitioner_x"].isnull()) & (lawyers_cases["petitioner_y"] == 1), "side"] = 'p'
lawyers_cases.loc[(lawyers_cases["petitioner_x"] == 1) & (lawyers_cases["other_y"] == 1), "side"] = 'p'
lawyers_cases.loc[(lawyers_cases["other_x"] == 1) & (lawyers_cases["petitioner_y"] == 1), "side"] = 'p'
lawyers_cases.loc[(lawyers_cases["amicus_curiae"] == 1) & (lawyers_cases["petitioner_y"] == 1), "side"] = 'p'

lawyers_cases.loc[(lawyers_cases["respondent_x"] == 1) & (lawyers_cases["respondent_y"] == 1), "side"] = 'r'
lawyers_cases.loc[(lawyers_cases["respondent_x"] == 1) & (lawyers_cases["respondent_y"].isnull()), "side"] = 'r'
lawyers_cases.loc[(lawyers_cases["respondent_x"].isnull()) & (lawyers_cases["respondent_y"] == 1), "side"] = 'r'
lawyers_cases.loc[(lawyers_cases["respondent_x"] == 1) & (lawyers_cases["other_y"] == 1), "side"] = 'r'
lawyers_cases.loc[(lawyers_cases["other_x"] == 1) & (lawyers_cases["respondent_y"] == 1), "side"] = 'r'
lawyers_cases.loc[(lawyers_cases["amicus_curiae"] == 1) & (lawyers_cases["respondent_y"] == 1), "side"] = 'r'

In [4]:
lawyers_cases = lawyers_cases[~((lawyers_cases['side'] == "NaN") & ~((lawyers_cases['other_x'] == 1) | (lawyers_cases['amicus_curiae'] == 1)))]

In [5]:
lawyers_traits = pd.DataFrame.from_csv("../data/advocate_data/lawyer_trait_predictions.csv")

In [6]:
lawyers_traits['path'] = lawyers_traits['path'].str.replace('.jpg', '')
lawyers_traits['path'] = lawyers_traits['path'].str.replace('_', ' ')
lawyers_traits.rename(columns={'path': 'name'}, inplace=True)

In [7]:
joined = lawyers_cases.merge(lawyers_traits,how='left',
         left_on=['name_merged'],
         right_on=['name'])

In [8]:
traits = list(joined.columns.values)[-40:]

In [9]:
df2 = joined.groupby(['term_merged', 'docket_merged','side'],as_index=False)[traits].mean()

In [10]:
# keep only certain columns
df2 = df2[["term_merged", "docket_merged", "side", "happy", "friendly", "caring", "unhappy", "sociable", "cold", "kind", "unemotional"]]

In [11]:
# df2[df2["side"] == "p"].to_csv("../data/advocate_data/petitioner_predictions.csv")

In [12]:
# df2[df2["side"] == "r"].to_csv("../data/advocate_data/respondent_predictions.csv")

In [13]:
pet = df2[df2["side"] == "p"]
pet = pet[pet['happy'].notnull()]

In [14]:
res = df2[df2["side"] == "r"]
res = res[res['happy'].notnull()]

In [15]:
pet.shape
# res.shape

(1483, 11)

In [23]:
final = pet.merge(res,how='outer',
        left_on=['term_merged','docket_merged'],
        right_on=['term_merged','docket_merged'])

In [28]:
final = final.fillna(final.mean()["happy_x":"unemotional_y"])

In [22]:
final.to_csv("../data/advocate_data/petitioner_and_respondent_ratings.csv")